In [1]:
import numpy as np
# IP import warnings because of "module 'numpy' has no attribute 'warnings'"
import warnings
np.warnings = warnings
from scipy import stats

# add the path to the abcTau package
import sys
#sys.path.append('./abcTau')
sys.path.append('C:\\Users\\ipochino\\.conda\\envs\\isttc\\Lib\\site-packages\\abcTau') # IP: replaced previous line with that; relative path was not working
import abcTau

In [2]:
abc_tau_folder = 'D:\\intr_timescales\\abcTau-master\\'
abc_tau_resuls = 'D:\\intr_timescales\\abcTau_results\\'

In [3]:
# path for loading and saving data
datasave_path = abc_tau_resuls + 'example_abc_results\\'
dataload_path = abc_tau_folder + 'example_data\\'

# path and filename to save the intermediate results after running each step
inter_save_direc = abc_tau_resuls + 'example_abc_results\\'
inter_filename = 'abc_intermediate_results'

# define filename for loading and saving the results
filename = 'OU_tau20_mean0_var1_rawData'
filenameSave = filename

In [4]:
# load data time-series as a numpy array (numTrials * time-points)
data_load = np.load(dataload_path + filename + '.npy')

# select summary statistics metric
summStat_metric = 'comp_ac_fft'
ifNorm = True # if normalize the autocorrelation or PSD

# extract statistics from real data
deltaT = 1 # temporal resolution of data.
binSize = 1 #  bin-size for binning data and computing the autocorrelation.
disp = None # put the dispersion parameter if computed with grid-search
maxTimeLag = 50 # only used when using autocorrelation for summary statistics
data_sumStat, data_mean, data_var, T, numTrials =  abcTau.preprocessing.extract_stats(data_load, deltaT, binSize,\
                                                                                  summStat_metric, ifNorm, maxTimeLag)

In [5]:
# select generative model and distance function
generativeModel = 'oneTauOU'
distFunc = 'linear_distance'

In [6]:
# Define a uniform prior distribution over the given range
# for a uniform prior: stats.uniform(loc=x_min,scale=x_max-x_min)
t_min = 0.0 # first timescale
t_max = 100.0
priorDist = [stats.uniform(loc= t_min, scale = t_max - t_min)]

In [7]:
# set fitting params
epsilon_0 = 1  # initial error threshold
min_samples = 100 # min samples from the posterior
steps = 60 # max number of iterations
minAccRate = 0.01 # minimum acceptance rate to stop the iterations
parallel = False # if parallel processing
n_procs = 1 # number of processor for parallel processing (set to 1 if there is no parallel processing)

In [8]:
# creating model object
class MyModel(abcTau.Model):

    #This method initializes the model object.  
    def __init__(self):
        pass

    # draw samples from the prior. 
    def draw_theta(self):
        theta = []
        for p in self.prior:
            theta.append(p.rvs())
        return theta

    # Choose the generative model (from generative_models)
    # Choose autocorrelation computation method (from basic_functions)
    def generate_data(self, theta):
        # generate synthetic data
        if disp == None:
            syn_data, numBinData =  eval('abcTau.generative_models.' + generativeModel + \
                                         '(theta, deltaT, binSize, T, numTrials, data_mean, data_var)')
        else:
            syn_data, numBinData =  eval('abcTau.generative_models.' + generativeModel + \
                                         '(theta, deltaT, binSize, T, numTrials, data_mean, data_var, disp)')
               
        # compute the summary statistics
        syn_sumStat = abcTau.summary_stats.comp_sumStat(syn_data, summStat_metric, ifNorm, deltaT, binSize, T,\
                                          numBinData, maxTimeLag)   
        return syn_sumStat

    # Computes the summary statistics
    def summary_stats(self, data):
        sum_stat = data
        return sum_stat

    # Choose the method for computing distance (from basic_functions)
    def distance_function(self, data, synth_data):
        if np.nansum(synth_data) <= 0: # in case of all nans return large d to reject the sample
            d = 10**4
        else:
            d = eval('abcTau.distance_functions.' +distFunc + '(data, synth_data)')        
        return d

In [9]:
# fit with aABC algorithm for any generative model
abc_results, final_step = abcTau.fit.fit_withABC(MyModel, data_sumStat, priorDist, inter_save_direc, inter_filename,\
                                                 datasave_path,filenameSave, epsilon_0, min_samples, \
                                                 steps, minAccRate, parallel, n_procs, disp)

Starting step 0
epsilon = 1
acceptence Rate = 1.0
--------------------
Starting step 1
epsilon = 0.14620652469927575
acceptence Rate = 0.704225352112676
--------------------
Starting step 2
epsilon = 0.10360220593592875
acceptence Rate = 0.684931506849315
--------------------
Starting step 3
epsilon = 0.06932948991383497
acceptence Rate = 0.6711409395973155
--------------------
Starting step 4
epsilon = 0.030105962309847547
acceptence Rate = 0.5208333333333334
--------------------
Starting step 5
epsilon = 0.017353103164245667
acceptence Rate = 0.5847953216374269
--------------------
Starting step 6
epsilon = 0.008842130811325455
acceptence Rate = 0.5076142131979695
--------------------
Starting step 7
epsilon = 0.005118946344941341
acceptence Rate = 0.4784688995215311
--------------------
Starting step 8
epsilon = 0.0030495707354353512
acceptence Rate = 0.5952380952380952
--------------------
Starting step 9
epsilon = 0.0018993793660718739
acceptence Rate = 0.5780346820809249
--------